In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt
import os

In [3]:
# Make a proper input file.
YAML="""
xopt: {output_path: temp, verbose: true}

algorithm:
  name: random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}

simulation:
  name: astra_with_generator
  function: astra.evaluate.evaluate_astra_with_generator
  options:
    astra_input_file: ../templates/kekgun/kekgun.in
    generator_input_file: ../templates/kekgun/dist004.in

# Note that you can call another file in the top level group:
vocs:
    variables:
      sig_x: [0.05, 1]
      lt: [0.005, 0.07]
      maxe(1): [20, 50]
      phi(1): [-30, 30]
      maxb(1): [0, 0.4]
      maxe(2): [0, 32]
      phi(2): [-180, 180]
      maxb(2): [0, 0.3]
      maxe(3): [0, 32]
      maxe(4): [0, 32]
      phi(3): [-45, 45]
      phi(4): [-45, 45]
      phi(6): [-45, 45]
    constants:
      ipart: 1000
      lspch: true
      zstop: 16.54

    objectives:
      end_core_emit_95percent_x: MINIMIZE
      end_sigma_z: MINIMIZE
    constraints:
      end_sigma_z: [LESS_THAN, 0.0015]
      end_core_emit_95percent_x: [LESS_THAN, 9.0e-07]
      end_sigma_energy: [LESS_THAN, 200000.0]
      end_higher_order_energy_spread: [LESS_THAN, 5000.0]
      end_mean_kinetic_energy: [GREATER_THAN, 90000000.0]
      end_n_particle_loss: [LESS_THAN, 1]
    linked_variables: null

"""

In [4]:
!mkdir temp

A subdirectory or file temp already exists.


In [5]:
# Create object
X = Xopt(config=YAML)

# Change some things to make it run faster
print(X.vocs)
X.vocs['constants']['lspch'] = True
X.vocs['constants']['ipart'] = 10
X.vocs['constants']['zstop'] = 0.2

# Show config
X

INFO:xopt.tools:Loading config from text
INFO:xopt.tools:Loading config from dict.
INFO:xopt.tools:Loading config from dict.
INFO:xopt.tools:Loading config from dict.
INFO:xopt.tools:Loading from YAML file: ../templates/kekgun/kekgun_vocs.yaml


{'variables': {'sig_x': [0.05, 1], 'lt': [0.005, 0.07], 'maxe(1)': [20, 50], 'phi(1)': [-30, 30], 'maxb(1)': [0, 0.4], 'maxe(2)': [0, 32], 'phi(2)': [-180, 180], 'maxb(2)': [0, 0.3], 'maxe(3)': [0, 32], 'maxe(4)': [0, 32], 'phi(3)': [-45, 45], 'phi(4)': [-45, 45], 'phi(6)': [-45, 45]}, 'constants': {'ipart': 1000, 'lspch': True, 'zstop': 16.54}, 'objectives': {'end_core_emit_95percent_x': 'MINIMIZE', 'end_sigma_z': 'MINIMIZE'}, 'constraints': {'end_sigma_z': ['LESS_THAN', 0.0015], 'end_core_emit_95percent_x': ['LESS_THAN', 9e-07], 'end_sigma_energy': ['LESS_THAN', 200000.0], 'end_higher_order_energy_spread': ['LESS_THAN', 5000.0], 'end_mean_kinetic_energy': ['GREATER_THAN', 90000000.0], 'end_n_particle_loss': ['LESS_THAN', 1]}, 'linked_variables': None}
{'xopt': {'output_path': 'temp', 'logging': 20}, 'algorithm': {'name': 'random_sampler', 'options': {'verbose': 'T', 'chunk_size': 4, 'max_samples': 10}, 'function': 'xopt.sampler.random_sampler', 'verbose': 'T', 'chunk_size': 4, 'max_s


            Xopt 
________________________________           
Version: 0.4.3+168.g7d48d79.dirty
Configured: True
Config as YAML:
xopt: {output_path: 'C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\xopt_class\temp',
  logging: 20}
algorithm:
  name: random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
  function: xopt.sampler.random_sampler
  verbose: T
  chunk_size: 4
  max_samples: 10
  executor: null
  output_path: null
simulation:
  name: astra_with_generator
  function: astra.evaluate.evaluate_astra_with_generator
  options: {astra_input_file: 'C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\templates\kekgun\kekgun.in',
    generator_input_file: 'C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\templates\kekgun\dist004.in',
    workdir: null, astra_bin: 'C:\Users\Ryan Roussel\Documents\bin\Astra', generator_bin: 'C:\Users\Ryan
      Roussel\Documents\bin\generator', timeout: 2500, verbose: false, auto_set_spacecharge_mesh: true,
    archive_path: n

In [6]:
# Check random inputs 
X.random_inputs()

{'sig_x': 0.4506418321156827,
 'lt': 0.01673721033001882,
 'maxe(1)': 24.297393195025546,
 'phi(1)': 7.782235762351391,
 'maxb(1)': 0.10805292560254305,
 'maxe(2)': 8.91182635230571,
 'phi(2)': -49.78463534333602,
 'maxb(2)': 0.2252859512992397,
 'maxe(3)': 7.201707565171198,
 'maxe(4)': 27.332043516888202,
 'phi(3)': 10.397233101525401,
 'phi(4)': -17.140073653049157,
 'phi(6)': -36.95594470933294,
 'ipart': 10,
 'lspch': True,
 'zstop': 0.2}

In [7]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

Wall time: 6.68 s


{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 1.6484e-06,
 'end_core_emit_95percent_x': 1.5181e-06,
 'end_core_emit_90percent_x': 1.3877999999999999e-06,
 'end_core_emit_80percent_x': 1.131e-06,
 'end_norm_emit_y': 1.9535e-06,
 'end_core_emit_95percent_y': 1.6844e-06,
 'end_core_emit_90percent_y': 1.4153999999999998e-06,
 'end_core_emit_80percent_y': 8.789599999999999e-07,
 'end_norm_emit_z': 14.29,
 'end_core_emit_95percent_z': 1.2474e-05,
 'end_core_emit_905percent_z': 1.0657e-05,
 'end_core_emit_80percent_z': 7.940999999999999e-06,
 'end_mean_t': 7.8486e-10,
 'end_mean_x': 0.0016778,
 'end_sigma_x': 0.0044137000000000004,
 'end_sigma_xp': 0.020472999999999998,
 'end_cov_x__xp': 9.03616801e-05,
 'end_mean_y': -0.0010809,
 'end_sigma_y': 0.0020902,
 'end_sigma_yp': 0.0096794,
 'end_cov_y__yp': 2.022352108e-05,
 'end_mean_kinetic_energy': 1287100.0,
 'end_sigma_z': 0.0034630999999999998,
 'end_sigma_energy': 12492.0,
 'end_cov_z__energy': -40.8334121,
 'end_n_particle_loss':

In [8]:
# These are the algorithm options
X.algorithm['options']

{'verbose': 'T', 'chunk_size': 4, 'max_samples': 10}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'astra_input_file': 'C:\\Users\\Ryan Roussel\\Documents\\GitHub\\xopt\\examples\\templates\\kekgun\\kekgun.in',
 'generator_input_file': 'C:\\Users\\Ryan Roussel\\Documents\\GitHub\\xopt\\examples\\templates\\kekgun\\dist004.in',
 'workdir': None,
 'astra_bin': 'C:\\Users\\Ryan Roussel\\Documents\\bin\\Astra',
 'generator_bin': 'C:\\Users\\Ryan Roussel\\Documents\\bin\\generator',
 'timeout': 2500,
 'verbose': False,
 'auto_set_spacecharge_mesh': True,
 'archive_path': None,
 'merit_f': None}

# Run Sampler using processes or threads

In [10]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [11]:
# Show config
X.run(executor)

INFO:xopt.xopt:Starting at time 2021-09-24T12:55:19-05:00




███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.3+168.g7d48d79.dirty

....4 samples completed in 0.11195 minutes
Samples written to:
 C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\xopt_class\temp\sampler-2021-09-24-1.json
....4 samples completed in 0.06209 minutes
Samples written to:
 C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\xopt_class\temp\sampler-2021-09-24-2.json
...

In [12]:
# Check for errors
X.results['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Write this configuration

In [13]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [14]:
!cat test.yaml

'cat' is not recognized as an internal or external command,
operable program or batch file.


# Run with MPI

In [15]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

'mpirun' is not recognized as an internal or external command,
operable program or batch file.


# Dask 

In [16]:
from dask.distributed import Client
executor = Client()
#executor = Client(processes=True)
executor

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65261,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 31.91 GiB
Comm: tcp://192.168.1.115:65291,Total threads: 4
Dashboard: http://192.168.1.115:65292/status,Memory: 7.98 GiB
Nanny: tcp://127.0.0.1:65266,


In [17]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [18]:
# Run some more
X.run(executor)
executor.close()

INFO:xopt.xopt:Starting at time 2021-09-24T12:55:40-05:00




███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.3+168.g7d48d79.dirty

....4 samples completed in 0.07485 minutes
Samples written to:
 C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\xopt_class\temp\sampler-2021-09-24-3.json
....4 samples completed in 0.05517 minutes
Samples written to:
 C:\Users\Ryan Roussel\Documents\GitHub\xopt\examples\xopt_class\temp\sampler-2021-09-24-4.json
...

In [19]:
X.results['error']

[False, False, False, True, False, False, False, False, False, False, False]

# Load into Pandas

In [20]:
import pandas as pd
from glob import glob
import json

In [21]:
jfiles = glob('temp/*json')
jfiles

['temp\\sampler-2021-09-23-1.json',
 'temp\\sampler-2021-09-23-10.json',
 'temp\\sampler-2021-09-23-11.json',
 'temp\\sampler-2021-09-23-12.json',
 'temp\\sampler-2021-09-23-13.json',
 'temp\\sampler-2021-09-23-14.json',
 'temp\\sampler-2021-09-23-15.json',
 'temp\\sampler-2021-09-23-16.json',
 'temp\\sampler-2021-09-23-17.json',
 'temp\\sampler-2021-09-23-18.json',
 'temp\\sampler-2021-09-23-19.json',
 'temp\\sampler-2021-09-23-2.json',
 'temp\\sampler-2021-09-23-20.json',
 'temp\\sampler-2021-09-23-21.json',
 'temp\\sampler-2021-09-23-22.json',
 'temp\\sampler-2021-09-23-23.json',
 'temp\\sampler-2021-09-23-24.json',
 'temp\\sampler-2021-09-23-25.json',
 'temp\\sampler-2021-09-23-26.json',
 'temp\\sampler-2021-09-23-27.json',
 'temp\\sampler-2021-09-23-28.json',
 'temp\\sampler-2021-09-23-3.json',
 'temp\\sampler-2021-09-23-4.json',
 'temp\\sampler-2021-09-23-5.json',
 'temp\\sampler-2021-09-23-6.json',
 'temp\\sampler-2021-09-23-7.json',
 'temp\\sampler-2021-09-23-8.json',
 'temp\\s

In [22]:
# Load a file
data = json.load(open(jfiles[0]))

data['inputs']
data['outputs']
data['error']

[False, False, False, False]

In [23]:
inputs = pd.DataFrame(data['inputs'])
inputs

,sig_x,lt,maxe(1),phi(1),maxb(1),maxe(2),phi(2),maxb(2),maxe(3),maxe(4),phi(3),phi(4),phi(6),ipart,lspch,zstop
0,0.683417,0.014690,36.703658,20.507834,0.072351,27.485161,33.737227,0.062214,6.599819,3.672196,13.086236,-43.878466,43.141303,1000,True,0.2
1,0.494340,0.042930,45.194041,14.475315,0.151047,9.346282,-81.493626,0.008933,12.137202,25.282354,7.229167,-33.976214,43.393058,1000,True,0.2
2,0.376364,0.055063,47.537885,-18.491361,0.320499,22.251986,21.791656,0.181640,2.462131,23.103032,3.756307,-38.096881,-24.595113,1000,True,0.2
3,0.182050,0.040702,22.101066,14.577752,0.262735,0.166122,-144.327526,0.148040,8.992023,4.068452,15.214028,-12.353572,2.018930,1000,True,0.2


In [24]:
outputs = pd.DataFrame(data['outputs'])
outputs

,error,end_mean_z,end_norm_emit_x,end_core_emit_95percent_x,end_core_emit_90percent_x,end_core_emit_80percent_x,end_norm_emit_y,end_core_emit_95percent_y,end_core_emit_90percent_y,end_core_emit_80percent_y,...,end_sigma_yp,end_cov_y__yp,end_mean_kinetic_energy,end_sigma_z,end_sigma_energy,end_cov_z__energy,end_n_particle_loss,end_total_charge,end_higher_order_energy_spread,fingerprint
0,False,0.2,6.407800e-07,4.268500e-07,3.530100e-07,2.505300e-07,6.289200e-07,4.274100e-07,3.551200e-07,2.504800e-07,...,0.001155,4.816504e-07,2076000.0,0.001718,37526.0,64.389389,0,9.990000e-11,1145.997378,6bcc49d64aee7f8a86800a09a3db277d
1,False,0.2,3.222300e-07,2.466000e-07,2.108000e-07,1.560100e-07,3.200700e-07,2.479700e-07,2.117400e-07,1.551700e-07,...,0.000316,3.273254e-08,2625100.0,0.004196,75280.0,311.550097,0,9.990000e-11,246.738609,f6d1706aea068a81e8408cdf7178fa94
2,False,0.2,2.676100e-07,2.015400e-07,1.709800e-07,1.247100e-07,2.638000e-07,2.017400e-07,1.707500e-07,1.235700e-07,...,0.000634,1.452577e-07,2729900.0,0.004029,92976.0,-368.812467,0,9.990000e-11,342.397245,bfe523fe8d2781ff7c408bba4100fe51
3,False,0.2,1.388700e-06,1.120500e-06,9.193300e-07,6.162900e-07,1.398700e-06,1.131700e-06,9.259400e-07,6.214300e-07,...,0.009349,1.970058e-05,1290100.0,0.004986,47899.0,235.754324,0,9.990000e-11,470.044435,4cc586af7cf1fa253bc9d7aeecc61ada


# Cleanup

In [25]:
import shutil
os.chmod('dask-worker-space/',0o777)
shutil.rmtree('dask-worker-space/')
os.chmod('test.yaml',0o777)
os.remove('test.yaml')